In [1]:
import sqlite3
from datetime import datetime, timedelta
import random
import uuid

In [2]:
def generate_random_name():
    first_names = ['Ravi', 'rahul', 'sandeep', 'pardeep', 'ishan', 'Shyra', 'khusboo', 'khushi', 'piyush', 'himsanshu']
    last_names = ['singh', 'goswami', 'Kumar', 'kumar', 'singh', 'singh', 'khan', 'salaria', 'kumar', 'bundela']
    return random.choice(first_names) + ' ' + random.choice(last_names)

In [3]:
def generate_random_phone_number():
    return f'+1({random.randint(100, 999)})-{random.randint(100, 999)}-{random.randint(1000, 9999)}'

In [4]:
def generate_random_prices():
    return [round(random.uniform(10, 1000), 2) for _ in range(730)]  # 2 years of data, 365 * 2 = 730 days

In [10]:
def generate_random_orders_and_items(num_orders, num_products, num_variants):
    orders = []
    order_items = []
    for i in range(num_orders):
        customer_id = random.randint(1, 10)  # Assuming you have 10 customers in the Customer table
        order_date = datetime(2021, 1, 1) + timedelta(days=random.randint(0, 730))  # Up to 2 years of order history
        order_id = str(uuid.uuid4())  # Generate a random UUID for order_id

        # Generate random order items for the order
        for j in range(random.randint(1, 5)):  # Each order can have 1 to 5 items
            product_id = random.randint(1, num_products)  # Assuming you have num_products products in the Product table
            variant_id = random.randint(1, num_variants)  # Assuming you have num_variants variants in the Variant table
            quantity = random.randint(1, 10)  # Random quantity between 1 and 10
            price = random.uniform(10, 1000)  # Random price between 10 and 1000

            order_items.append((order_id, product_id, variant_id, quantity, price))

        orders.append((order_id, customer_id, order_date))

    return orders, order_items

In [11]:
conn = sqlite3.connect('ecommerce4.db')
cursor = conn.cursor()

In [12]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    shipping_address TEXT,
    contact_number TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Variant (
    variant_id INTEGER PRIMARY KEY,
    attribute1 TEXT,
    attribute2 TEXT,
    product_id INTEGER,
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
)
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS PriceHistory (
    price_id INTEGER PRIMARY KEY,
    entity_id INTEGER,
    entity_type TEXT,
    price REAL,
    effective_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    order_id TEXT PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS OrderItems (
    order_item_id INTEGER PRIMARY KEY,
    order_id TEXT,
    product_id INTEGER,
    variant_id INTEGER,
    quantity INTEGER,
    price REAL,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id),
    FOREIGN KEY (variant_id) REFERENCES Variant(variant_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS AddressHistory (
    address_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    address TEXT,
    effective_date DATE,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
)
''')

In [14]:
def generate_random_address():
    streets = ['Main Street', 'Park Avenue', 'Elm Street', 'Broadway', 'Oak Lane', 'Cedar Avenue']
    cities = ['Amritsar', 'Jalandhar', 'Jais', 'Raebarilei', 'chandighar', 'Phoenix']
    states = ['PB', 'PB', 'UP', 'UP', 'UP', 'CS']
    postal_codes = ['10001', '90001', '60601', '77001', '19101', '85001']

    street = random.choice(streets)
    city = random.choice(cities)
    state = random.choice(states)
    postal_code = random.choice(postal_codes)

    return f"{street}, {city}, {state} {postal_code}"

In [15]:
for entity_type, entity_id in [("Product", 1), ("Product", 2)]:
    prices = generate_random_prices()
    start_date = datetime(2021, 1, 1)
    price_history = []
    for price in prices:
        effective_date = start_date.strftime('%Y-%m-%d')
        price_history.append((entity_id, entity_type, price, effective_date))
        start_date += timedelta(days=1)
    cursor.executemany('INSERT INTO PriceHistory (entity_id, entity_type, price, effective_date) VALUES (?, ?, ?, ?)', price_history)

In [16]:
orders, order_items = generate_random_orders_and_items(100, 10, 10)
for order in orders:
    order_id = str(uuid.uuid4())  # Generate a random UUID for order_id
    cursor.execute('INSERT INTO Orders (order_id, customer_id, order_date) VALUES (?, ?, ?)', (order_id, order[1], order[2]))

cursor.executemany('INSERT INTO OrderItems (order_id, product_id, variant_id, quantity, price) VALUES (?, ?, ?, ?, ?)', order_items)

In [17]:
conn.commit()
conn.close()